<a href="https://colab.research.google.com/github/TasbihElyamani/MLOps/blob/main/MLOPs_LifeCycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pandas scikit-learn joblib fastapi uvicorn prometheus_client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
df = pd.read_csv(url)

# Split dataset
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Save processed data
train.to_csv("./train.csv", index=False)
test.to_csv("./test.csv", index=False)

print("Data preparation completed. Train and Test data saved.")


Data preparation completed. Train and Test data saved.


In [6]:
import pandas as pd
import joblib
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load training data
train_data = pd.read_csv("./train.csv")

# Features and target
X_train = train_data.drop(columns=["medv"])  # Target column: 'medv'
y_train = train_data["medv"]

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Save model with versioning
version = 1
model_path = f"./model_v{version}.pkl"

while os.path.exists(model_path):
    version += 1
    model_path = f"./model_v{version}.pkl"

joblib.dump(model, model_path)
print(f"Model saved: {model_path}")


Model saved: ./model_v1.pkl


In [7]:
import joblib
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

# Load latest model
model_path = "./model_v1.pkl"  # Change this based on the latest version
model = joblib.load(model_path)

class HouseFeatures(BaseModel):
    crim: float
    zn: float
    indus: float
    chas: int
    nox: float
    rm: float
    age: float
    dis: float
    rad: int
    tax: float
    ptratio: float
    b: float
    lstat: float

@app.post("/predict")
def predict_price(features: HouseFeatures):
    data = pd.DataFrame([features.dict().values()])
    prediction = model.predict(data)[0]
    return {"predicted_price": prediction}

# Run API using `uvicorn scripts.deploy:app --reload`


In [ ]:
from prometheus_client import start_http_server, Summary
import time
import random

# Create Prometheus metrics
PREDICTION_TIME = Summary("prediction_processing_seconds", "Time spent processing predictions")

@PREDICTION_TIME.time()
def process_prediction():
    time.sleep(random.uniform(0.1, 0.5))  # Simulating prediction time

if __name__ == "__main__":
    start_http_server(8001)  # Prometheus will scrape metrics from this port
    while True:
        process_prediction()
        time.sleep(2)
